In [4]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing

#1) READ AND EXTRACT DATA

#1.1) get data out of the csv file
dataframe=pandas.read_csv("./RealMedicalData2.csv",sep=';',decimal=b',')

listColNames=list(dataframe.columns)


#1.2) extract X and Y as numpy arrays

XY=dataframe.values
ColNb_Y=listColNames.index('Treatment')


Y=XY[:,ColNb_Y].reshape((XY.shape[0],1))   #reshape is to make sure that Y is a column vector
X = np.delete(XY, ColNb_Y, 1)

X_scaled = preprocessing.scale(X)

listColNames.pop(ColNb_Y)     #to make it contains the column names of X only

#2) EXPLORE THE DATA
"""
for Col in range(len(listColNames)):
    plt.plot(X[:,Col],Y[:],'.')
    plt.xlabel(listColNames[Col])
    plt.ylabel('Disease progression')
    plt.show()
"""
#3) PERFORM THE REGRESSION

from sklearn.linear_model import Ridge
logisticRegr = LogisticRegression()
c,i,n=logisticRegr.fit(X_scaled,Y)
print(c,i,n)


NameError: name 'LogisticRegression' is not defined

### QUESTION 1: 
Ouvrir le fichier RealMedicalData.csv (par exemple avec libreoffice) puis comprendre 
            chaque instruction du code

In [4]:
#ok

### QUESTION 2: 
Trouver une bonne valeur de alpha pour la regression Ridge et Lasso en utilisant la
            separant les observations en un jeu d'apprentissage et un jeu de validation
              -> Est-ce que les deux méthodes ont un bon pouvoir de prediction (utiliser metrics.r2_score) ?

In [18]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

kf = KFold(n_splits=3)
alpha_lst=[0.0000001,0.0001,0.001,0.01,0.1,1,10,100,1000,100000] #recherche grossiere
#alpha_lst=[0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01] #recherche grossiere
#alpha_lst=[0.08]
for k,alpha in enumerate(alpha_lst):
    r2_lst,r2_ridge_lst=[],[]
    for train, test in kf.split(X_scaled):
        
        X_train = X_scaled[train]
        y_train = Y[train]
        X_test = X_scaled[test]
        y_test = Y[test]

        ridge_regressor=Ridge(alpha=alpha, fit_intercept=True)
        ridge_regressor.fit(X_train, y_train)
        
        lasso_regressor=Lasso(alpha=alpha, fit_intercept=True)
        lasso_regressor.fit(X_train, y_train)
        
        
        y_pred_lasso = lasso_regressor.predict(X_test)
        r2_score_lasso = mean_squared_error(y_test, y_pred_lasso)
        r2_lst.append(r2_score_lasso)
        
        y_pred_ridge = ridge_regressor.predict(X_test)
        r2_score_ridge = mean_squared_error(y_test, y_pred_ridge)
        r2_ridge_lst.append(r2_score_ridge)
    
    print(np.mean(r2_lst))
    print(np.mean(r2_ridge_lst))
    print('------'+str(alpha)+'------')

41.5208375547
41.5208414722
------1e-07------
41.5162688847
41.5201659614
------0.0001------
41.4759061967
41.5140844181
------0.001------
41.0814588916
41.4536722769
------0.01------
32.958866514
40.8843324957
------0.1------
15.2312879815
36.7548464803
------1------
14.220066841
24.1892134258
------10------
14.220066841
16.6184393247
------100------
14.220066841
14.5715834794
------1000------
14.220066841
14.2237497228
------100000------


### QUESTION 3: 
Afin de comprendre le lien entre l'evolution de la maladie et les variables etudiees,
            on sélectionne tout au plus 3 variables avec le lasso.
            -> Utiliser une procedure de type 4-folds pour selectionner typiquement 3 variables
            -> Les variables selectionnees sont elles stables ?
            -> Est-ce que le modele garde un bon pouvoir de prediction lorsque avec 3 variables selectionnees


### QUESTION 4:
Eventuellement, tester un algorithme de selection de type forward avec un critere BIC. 
Comparer les variables selectionnees avec celles de la question 3